In [ ]:
# 方言翻译数据集合准备


# 基本流程
    - 抽取中文对话数据集
    - 统计出现最多的词
    - 建立词表
    - 人工标注
    - 简单替换词翻译
    - 检查翻译结果

## Step1 中文对话数据集合

In [ ]:
import json

from datasets import Dataset

input_file = "./train.jsonl"


def load_jsonl_data(file_path):
    json_data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            json_data.append(json.loads(line.strip()))
    return json_data


# 将数据加载为 Dataset 格式
data = load_jsonl_data(input_file)
dataset = Dataset.from_list(data)
dataset

In [ ]:
questions = dataset['question']

In [ ]:
answers = dataset['answer']

# Step2 统计出现最多的词

In [ ]:
import jieba, re
from collections import Counter

# 将所有标题合并为一个长字符串
text = " ".join(questions)

# 使用 jieba 进行分词
words = jieba.lcut(text)

# 使用 Counter 统计词频
# 去掉标点符号并过滤掉单字词
filtered_words = [word for word in words if len(word) > 1 and re.match(r'[\u4e00-\u9fff]+', word)]

# 使用 Counter 统计词频
word_counts = Counter(filtered_words)

# 找出出现次数最多的词
most_common_words = word_counts.most_common(200)  # 获取出现次数最多的200个词

# 写入CSV
print("出现次数最多的词：")
with open('most_common_words.csv', 'w', encoding='utf-8') as f:
    for word, count in most_common_words:
        print(f"{word},{count}")
        f.write(f"{word},{count}\n")

# Step3 人工标注
## 因为没有相关的翻译数据集，只能将常用的词人工翻译。

# Step4根据人工标注数据，简单替换词翻译

In [ ]:
with open('translate_dict.csv', 'r', encoding='gbk') as f:
    lines = f.readlines()
    translate_dict = {line.split(',')[1].strip(): line.split(',')[0].strip() for line in lines}
    # 根据字符长度排序
    translate_dict = sorted(translate_dict.items(), key=lambda x: len(x[0]), reverse=True)

translate_dict

In [ ]:
# Step5简单翻译数据集

In [ ]:
def translate(trans_text):
    total_replaced_chars = 0  # 记录替换的字数

    for k, v in translate_dict:
        while k in trans_text:
            # 记录替换前后长度的差值（即替换的字数）
            replaced_count = trans_text.count(k) * len(k)
            total_replaced_chars += replaced_count

            # 进行替换
            trans_text = trans_text.replace(k, v)

    return trans_text, total_replaced_chars

In [ ]:
trans_data = []

# 翻译结果写入为jsonl
for question, answer in zip(questions, answers):
    trans_question, question_replaced = translate(question)
    if question != trans_question:
        sc = question_replaced / len(question)
        if sc > 0.2:
            trans_data.append({"input_text": question, "target_text": trans_question, "replaced_chars": question_replaced})

    trans_answer, answer_replaced = translate(answer)
    if answer != trans_answer:
        if question_replaced / len(question) > 0.2:
            trans_data.append({"input_text": answer, "target_text": trans_answer, "replaced_chars": answer_replaced})

# 随机打乱数据集
import random

random.shuffle(trans_data)
with open('translate_trans_data.jsonl', 'w', encoding='utf-8') as f:
    for item in trans_data[:40000]:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')